# Feedforward Networks for Handwritten Digit Recognition

In this assignment you will learn how to use feedforward neural networks to solve a classical task in machine learning: handwritten digit recognition using images from the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). More concretely, you will have to solve the following tasks:

1. implement a deep feedforward network that reads a batch of images and predicts the corresponding digits
2. train this network using stochastic gradient descent

In order to run the code for this assignment, you need a working installation of [NumPy](http://www.numpy.org). Check whether everything works by running the following code cell:

In [1]:
import numpy as np

## Data

The MNIST dataset is split into a training set with 60,000 instances and a test set with 10,000 instances. Each instance consists of a greyscale image of a handwritten digit and an integer representing the digit in the image, as labelled by human experts. The digits are scaled and centred on a 28-by-28 pixel canvas. Here are some examples of how the images can look like:

![The first handwritten digits in the MNIST training data](images/mnist.png)

The following code will read the training data and the test data:

In [2]:
import mnist_network

training_x, training_y = mnist_network.read_training_data()
print('Shapes of the training data matrices: {} {}'.format(training_x.shape, training_y.shape))

test_x, test_y = mnist_network.read_test_data()
print('Shapes of the test data matrices: {} {}'.format(test_x.shape, test_y.shape))

Shapes of the training data matrices: (60000, 784) (60000, 10)
Shapes of the test data matrices: (10000, 784) (10000, 10)


From a Python perspective, each of the two data sets is a pair `(x, y)` of matrices: Each row of `x` is a 784-component vector containing the greyscale values of the pixels in an image as floats between 0 and 1. Each row of `y` is a 10-component one-hot vector representing the digit corresponding to the image. As an example, here is the vector for the first digit in the test data, the digit 7:

In [3]:
test_y[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

## Task 1: Implement the network

Your first task is to implement a deep feedforward network that reads a batch of image vectors and predicts the corresponding digits. Your network should conform to the following specification:

* one input layer, one output layer, flexible number of hidden layers
* activation function for hidden layers: rectified linear unit (ReLU)
* activation function for output layer: softmax
* error function for gradient computation: categorical cross-entropy

To get you started on this task, we provide skeleton code and a number of useful helper functions.

### Helper functions

The following cell contains NumPy-based implementations of the ReLU activation function and its derivative (which you should use for the hidden layers of your network), as well as the softmax activation function (for the output layer).

In [4]:
def relu(x):
    return x * (x > 0)

def relu_prime(x):
    return 1 * (x > 0)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

In each case, the argument `x` is a batch of input values, such as `training_x`. The implementation of the softmax function uses a standard trick to improve numerical stability; see [this link](http://stackoverflow.com/questions/34968722/softmax-function-python) for more information.

### Skeleton code

To get you started, we provide the following skeleton code:

In [5]:
class Network():

    def __init__(self, sizes):
        self.ws = [np.random.randn(m,n) / np.sqrt(n / 2) for m, n in zip(sizes[:-1], sizes[1:])]
        self.bs = [np.zeros((1, n)) for n in sizes[1:]]
        self.hidden_layer_num = len(sizes) - 2
        # for optimization
        self.ws_m = [np.zeros((m, n)) for m, n in zip(sizes[:-1], sizes[1:])]
        self.ws_v = [np.zeros((m, n)) for m, n in zip(sizes[:-1], sizes[1:])]
        self.bs_v = [np.zeros((1, n)) for n in sizes[1:]]

    def forward(self, x):
        unit_output = []
        unit_output.append(x)
        weigthed_sum = []
        for i in range(self.hidden_layer_num):
            unit_output.append(relu(np.dot(x, self.ws[i]) + self.bs[i]))
            weigthed_sum.append(np.dot(x, self.ws[i]) + self.bs[i])
            x = relu(np.dot(x, self.ws[i]) + self.bs[i])
        x = softmax(np.dot(x, self.ws[-1]) + self.bs[-1])
        return x, unit_output, weigthed_sum

    def predict(self, x):
        prediction = self.forward(x)[0]
        return np.argmax(prediction, axis = 1)

    def update_weight(self, weight, lr, delta, y, i, opt_option = 0):
        if opt_option is 0:
            #without optimization
            return weight - lr * np.dot(y, delta)
        else:
            #L2 regularision
            alpha = 0.001
            gradient = np.dot(y, delta) + alpha * weight
            #Adam
            beta_1 = 0.9
            beta_2 = 0.999
            eps = 1e-8
            self.ws_m[i] = beta_1 * self.ws_m[i] + (1 - beta_1) * gradient
            self.ws_v[i] = beta_2 * self.ws_v[i] + (1 - beta_2) * gradient * gradient
            hidden_m = self.ws_m[i] / (1 - pow(beta_1, i + 1))
            hidden_v = self.ws_v[i] / (1 - pow(beta_2, i + 1))
            return weight - lr * hidden_m / (np.sqrt(hidden_v) + eps)

    def update_bias(self, bias, lr, delta, i, opt_option = 0):
        if opt_option is 0:
            #without optimization
            return bias - lr * delta.sum(axis = 0, keepdims = True)
        else:
            #RMSProp
            beta = 0.9
            eps = 1e-8
            gradient = delta.sum(axis = 0, keepdims = True)
            self.bs_v[i] = beta * self.bs_v[i] + (1 - beta) * gradient * gradient
            return bias - lr * gradient / (np.sqrt(self.bs_v[i] + eps))
    
    def delta_cross_entropy(self, X, y):
            y = np.argmax(y, axis = 1)
            m = y.shape[0]
            grad = X
            grad[range(m), y] -= 1
            grad = grad/m
            return grad
        
    def backpropagate(self, x, y, lr = 1e-3, optimizer = 0):
        forward_output, unit_output, weighted_sum = self.forward(x)
        delta = self.delta_cross_entropy(forward_output, y)
        output_index = self.hidden_layer_num
        #update output layer's weights and bias
        self.ws[output_index] = self.update_weight(self.ws[output_index], lr, delta, unit_output[output_index].T, output_index, optimizer)
        self.bs[output_index] = self.update_bias(self.bs[output_index], lr, delta, output_index, optimizer)
        #update hidden layers' weights and bias
        for i in range(self.hidden_layer_num, 0, -1):
            hidden_derviate = relu_prime(weighted_sum[i-1])
            delta = hidden_derviate * np.dot(delta, self.ws[i].T)
            self.ws[i-1] = self.update_weight(self.ws[i-1], lr, delta, unit_output[i-1].T, i-1, optimizer)
            self.bs[i-1] = self.update_bias(self.bs[i-1], lr, delta, i-1, optimizer)

This code defines a class Network that represents deep feedforward networks. The intended behaviour of the fields and methods of this class is specified below. In the skeleton code, all fields are initialized with, and all methods return, zero matrices of the appropriate shape. In order to obtain a network that meets the requirements, you will have to write code that replaces these placeholders with meaningful values.

In your implementation, you may choose to add more fields and/or methods than the ones included in the skeleton code. However, in all of your code, you may only call functions from the NumPy library, but no other library.

### Fields

**sizes : list(int)**

The dimensions of the network layers, from the first (input) layer to the last (output) layer.

An example, in a network with 784 units in the input layer, 10 units in the output layer, and 100 units in the (single) hidden layer this field would have the value

In [6]:
[784, 100, 10]

[784, 100, 10]

###### **ws : list(np.array)**

The weight matrices of the network, where the matrix at index $i$ holds the weights of the connections from layer $i$ to layer $i+1$. As an example, if the width of layer&nbsp;0 is $784$ and the width of layer&nbsp;1 is $100$, then `ws[0]` will have shape $(784, 100)$.

**bs : list(np.array)**

The bias vectors of the network, where the vector at index $i$ holds the biases for layer $i+1$. As an example, `bs[0]` is the bias vector of layer&nbsp;1. Note that there are no biases for the input layer (layer&nbsp;0).

### Initialization

Initialize the weights and biases of the network. Note that in the starter code, both weights and biases are initialized using zeros.

**sizes : list(int)**

The dimensions of the network layers, from the first (input) layer to the last (output) layer.

As an example, the following code creates a network with 784 units in the input layer, 10 units in the output layer, and 100 units in the (single) hidden layer:

In [7]:
net = Network([784, 100, 10])

### forward

Computes the output of the network for a batch of input values.

**x : np.array**

A batch of input values, such as `training_x`.

**Returns:** The output of the network for the specified input. This will be an array of shape $(m, n)$ where $m$ is the number of rows in the input batch, and $n$ is the size of the last layer in the network. In the starter code, the method returns an array of all zeros.

### predict

Predicts the digits for a batch of input values.

**x : np.array**

A batch of input values, such as `test_x`.

**Returns:** The digits predicted for the specified input. This will be an array of shape $(m, 1)$ where $m$ is the number of rows in the input batch $x$. In the starter code, the method returns an array of all zeros.

### backpropagate

Computes the network gradients for a batch of input and corresponding output values. Note that in the context of this assignment, the gradients should be computed relative to categorical cross-entropy as the error function.

During backpropagation, you will have to compute the derivative of the cross-entropy error function with respect to the softmax *input* $z$. While you could do that by multiplying the derivative of the cross-entropy error function with respect to the softmax *output* and the derivative of the softmax function with respect to its input, this particular combination has a very simple form: $t - z$, where $t$ is the (one-hot) target output vector. See [this page](http://peterroelants.github.io/posts/cross_entropy_softmax/) for a derivation.

**x : np.array**

A batch of input values, such as `training_x`.

**y : np.array**

A batch of corresponding output values, such as `training_y`.

**Returns:** A list of pairs of the form $(\nabla w, \nabla b)$, one for each non-input layer of the network, where the first component of each pair is the average gradient for the weights of the connections coming into the layer and the second component is the average gradient for the biases at the layer. In the starter code, the method returns a list of zero gradients.

## Task 2: Train your network

Once you have completed the Network class, your second task is to write code to train the network using stochastic gradient descent (SGD).

### Helper functions

The function in the next code cell will sample minibatches from an array `x` of input values and a corresponding array `y` of output values:

In [8]:
def minibatches(x, y, batch_size):
    random_indices = np.random.permutation(np.arange(x.shape[0]))
    for i in range(0, x.shape[0] - batch_size + 1, batch_size):
        batch_indices = random_indices[i:i+batch_size]
        yield x[batch_indices], y[batch_indices]

The next function computes the test error rate of a network on a batch of test data.

In [9]:
def evaluate(net):
    return np.mean(net.predict(test_x) != np.argmax(test_y, axis=1))

### Skeleton code

The following cell contains skeleton code for the training algorithm.

In [10]:
def train_sgd(net, x, y, n_epochs, batch_size, eta=1e-3, optimizer = 0):
    for t in range(n_epochs):
        for batch_x, batch_y in minibatches(x, y, batch_size):
            # TODO: Do something useful here
            net.backpropagate(batch_x, batch_y, eta, optimizer)
        print("epoch = {}, test error rate = {:.4f}".format(t, evaluate(net)))

The intended meaning of the various parameters is as follows:

**x : np.array**

A batch of input values, such as `training_x`.

**y : np.array**

A batch of corresponding output values, such as `training_y`.

**n_epochs : int**

The number of iterations over the training data (&lsquo;epochs&rsquo;).

**batch_size : int**

The number of input values per minibatch.

**eta : float**

The learning rate in the stochastic gradient descent update step.

### Intended usage

To see how the training code is intended to be used, here is how you set up a network and train it on the training data for 2&nbsp;iterations with minibatch size&nbsp;30 and the default learning rate.

In [11]:
net = Network([784, 1500, 10])
train_sgd(net, training_x, training_y, 10, 15, 1e-2, 0)

epoch = 0, test error rate = 0.0381
epoch = 1, test error rate = 0.0265
epoch = 2, test error rate = 0.0234
epoch = 3, test error rate = 0.0211
epoch = 4, test error rate = 0.0206
epoch = 5, test error rate = 0.0205
epoch = 6, test error rate = 0.0193
epoch = 7, test error rate = 0.0185
epoch = 8, test error rate = 0.0188
epoch = 9, test error rate = 0.0185


In [14]:
net = Network([784, 1000,1000, 10])
train_sgd(net, training_x, training_y, 10, 500, 1e-3, 1)

epoch = 0, test error rate = 0.0381
epoch = 1, test error rate = 0.0290
epoch = 2, test error rate = 0.0257
epoch = 3, test error rate = 0.0242
epoch = 4, test error rate = 0.0220
epoch = 5, test error rate = 0.0225
epoch = 6, test error rate = 0.0210
epoch = 7, test error rate = 0.0217
epoch = 8, test error rate = 0.0199
epoch = 9, test error rate = 0.0200


In [16]:
net = Network([784, 1000,1000, 10])
train_sgd(net, training_x, training_y, 10, 200, 1e-3, 1)

epoch = 0, test error rate = 0.0345
epoch = 1, test error rate = 0.0247
epoch = 2, test error rate = 0.0235
epoch = 3, test error rate = 0.0208
epoch = 4, test error rate = 0.0193
epoch = 5, test error rate = 0.0196
epoch = 6, test error rate = 0.0202
epoch = 7, test error rate = 0.0204
epoch = 8, test error rate = 0.0211
epoch = 9, test error rate = 0.0205


In [17]:
net = Network([784, 1000,800, 10])
train_sgd(net, training_x, training_y, 10, 500, 1e-3, 1)

epoch = 0, test error rate = 0.0421
epoch = 1, test error rate = 0.0276
epoch = 2, test error rate = 0.0259
epoch = 3, test error rate = 0.0233
epoch = 4, test error rate = 0.0215
epoch = 5, test error rate = 0.0213
epoch = 6, test error rate = 0.0210
epoch = 7, test error rate = 0.0196
epoch = 8, test error rate = 0.0196
epoch = 9, test error rate = 0.0194


In [20]:
net = Network([784, 1000,800,600, 10])
train_sgd(net, training_x, training_y, 10, 300, 1e-3, 1)

epoch = 0, test error rate = 0.0367
epoch = 1, test error rate = 0.0310
epoch = 2, test error rate = 0.0270
epoch = 3, test error rate = 0.0234
epoch = 4, test error rate = 0.0219
epoch = 5, test error rate = 0.0226
epoch = 6, test error rate = 0.0208
epoch = 7, test error rate = 0.0214
epoch = 8, test error rate = 0.0196
epoch = 9, test error rate = 0.0213


### Performance goal

Once you have a working network and training algorithm, you can compare the error rate of your network to the results on the [MNIST website](http://yann.lecun.com/exdb/mnist/).

**To get credit for this assignment, your network must achieve a test error rate of less than 2% at least once during the first 10 epochs of training.**

To tune your network, you can play around with the various training parameters: number of epochs, minibatch size, and learning rate. In addition to that, you can also make more substantial changes such as the following:

* Make the network wider (increase the size of a layer) or deeper (add more layers).
* Implement a different initialization strategy.
* Implement a regularization method or dropout.
* Implement an optimization algorithm with an adaptive learning rate, such as RMSProp or Adam.

## How to submit

When you have reached the performance goal, send this notebook to Marco to receive credit for the assignment. The notebook must be self-contained and must run without error.

In addition to your code, you are asked to submit a short text (less than 500&nbsp;words) in which you reflect on what you have done. Which specific choices did you make when tuning your network? How did these choices affect performance? You can enter your text in the cell below.

### Report

I started to program for this lab after reading GBC chapter 6. I just initialized the weights and bias as zero. So the first tries to train the net had very higher error rate. After I read chapters 7 and 8. I realized that such initialization for weights was not reasonable, so I changed it into a normalized initialization. This helps a lot, the net can reach an error rate less than 2% with only 1 hidden layer,  1500 hidden units, 15 mini batch size and 1e-2 learning rate. Then I try to increase the number of hidden layers into 2 to ensure the whole code is correct and to meet the performance goal of this lab also. However, after several steps in the training, the 'divide by zero' problem happens. Then I found the weights go to extremly high during the training which is result from gradient explodsion. I realized that I should use some optimizatized methods. My first try is to use L2 regularision and RMSProp together. The error rate decreases reasonably but reaches to less than 2% rarely. 
Then I used Adam method instead of RMSProp. 

*Good luck!*